<a href="https://colab.research.google.com/github/dai2-cell/cis-5450/blob/main/Module_1_Data_Acquisition_Wrangling_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I of Big Data Analytics - Notebook 2

## Recall

As we start our journey into Big Data Analytics, the first thing we need to do is **get the data** in the form we need for analysis!  We'll start with an overview of how to acquire and *wrangle* data.

This notebook will be built incrementally to consider several tasks:

* Acquiring data from files and remote sources
* Information extraction over HTML content
* A basic "vocabulary" of operators over tables (the relational algebra)
* Basic manipulation using SQL in DuckDB

* "Data wrangling" or integration:
  * Cleaning and filtering data, using rules and based operations
  * Linking data across dataframes or relations
  * The need for approximate match and record linking
  * Different techniques

## Before you start this notebook

Make sure you did the exercises in [Notebook 1](https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/Module_1_Data_Acquisition.ipynb)!

## The Motivating Question
To illustrate the principles, we focus on the question of **how old company CEOs and founders** (in general, leaders) are.  The question was in part motivated by the following New York Times article:

* Founders of Successful Tech Companies Are Mostly Middle-Aged: https://www.nytimes.com/2019/08/29/business/tech-start-up-founders-nest.html?searchResultPosition=2

So let's test this hypothesis!

## Initial Libraries

We'll be using [DuckDB](https://duckdb.org/) as a means of managing our tables.  DuckDB works like a Python library, but manages a full SQL database (in files).  It also integrates very nicely with Pandas, so we'll use it in this course.

In [1]:
!pip3 install duckdb

In [2]:
!pip3 install lxml

In [3]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [4]:
!pip3 install penngrader-client

For quiz credit you'll need to update your student ID here!

In [5]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 18275381 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

Quizzes will cumulatively count as HW9... Don't edit this...

In [6]:
%set_env HW_ID=cis5450_25f_HW9

env: HW_ID=cis5450_25f_HW9


In [7]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

PennGrader initialized with Student ID: 18275381

Make sure this correct or we will not be able to store your grade


In [8]:
# Imports we'll use through the notebook, collected here for simplicity

# For parsing dates and being able to compare
import datetime

# For fetching remote data
import urllib
import urllib.request

# Pandas dataframes and operations
import pandas as pd

# Numpy matrix and array operations
import numpy as np

# Sqlite is a simplistic database
import duckdb

# Data visualization
import matplotlib

from lxml import etree

# 1. Reload from Module Notebook 1

Here is a quick reload of the data from Module Notebook 1.

For simplicity, we reload all of the following:
* `company_ceos_df` -- Wikipedia list of companies and their CEOs
* `exec_df` -- crawled executive names and birthdays
* `company_data_df` -- 7M entries about companies
* `company_info_df` -- CSV about companies and lines of business

In [9]:
!wget -nc https://storage.googleapis.com/penn-cis5450/companies_sorted.csv

--2025-09-16 15:08:23--  https://storage.googleapis.com/penn-cis5450/companies_sorted.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.207, 74.125.196.207, 173.194.217.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1085578742 (1.0G) [text/csv]
Saving to: ‘companies_sorted.csv’

companies_sorted.cs 100%[===================>]   1.01G  85.4MB/s    in 15s     

2025-09-16 15:08:38 (71.0 MB/s) - ‘companies_sorted.csv’ saved [1085578742/1085578742]



Here's a helper for importing HTML

In [10]:
import requests
import pandas as pd
from io import StringIO

headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def import_html(url):
  # Let's read an HTML table!
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }
  page = requests.get(url, headers=headers).text
  return page


In [11]:
from io import StringIO

company_ceos_df = pd.read_html(StringIO(import_html('https://en.wikipedia.org/wiki/List_of_chief_executive_officers#List_of_CEOs')))[1]

pages = []
page_urls = []

for i, executive in enumerate(company_ceos_df['Executive']):
  url = 'https://en.wikipedia.org/wiki/' + executive.replace(' ', '_')

  page = url.split("/")[-1] #extract the person name at the end of the url

  # An issue: some of the accent characters won't work.  We need to convert them
  # into an HTML URL.  We'll split the URL, then use "parse.quote" to change
  # the structure, then re-form the URL
  url_list = list(urllib.parse.urlsplit(url))
  url_list[2] = urllib.parse.quote(url_list[2])
  url_ascii = urllib.parse.urlunsplit(url_list)
  try:
    response = import_html(url_ascii)
    pages.append(response)
    page_urls.append(url)
  except requests.exceptions.RequestException as e:
    print(e)


# Use lxml.etree.HTML(...) on the HTML content of each page to get a DOM tree that
# can be processed via XPath to extract the bday information.  Store the CEO name,
# webpage, and the birthdate (born) in exec_df.

# We first check that the HTML content has a table of type `vcard`,
# and then extract the `bday` information.  If there is no birthdate, the datetime
# value is NaT (not a type).

rows = []
for i, page in enumerate(pages):
  url = page_urls[i]

  tree = etree.HTML(page)  #create a DOM tree of the page
  bday = tree.xpath('//table[contains(@class,"vcard")]//span[@class="bday"]/text()')
  if len(bday) > 0:
      name = url[url.rfind('/')+1:] # The part of the URL after the last /
      rows.append({'name': name, 'page': url,
                  'born': datetime.datetime.strptime(bday[0], '%Y-%m-%d')})
  else:
          rows.append({'name': url[url.rfind('/')+1:], 'page': url
                                    , 'born': np.datetime64('NaT')})

exec_df = pd.DataFrame(rows)
exec_df['clean_name'] = exec_df['name'].apply(lambda x: x.replace('_', ' '))
company_data_df = pd.read_csv('companies_sorted.csv')
countries_df = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

data = urllib.request.urlopen(\
       'https://gist.github.com/jvilledieu/c3afe5bc21da28880a30/raw/a344034b82a11433ba6f149afa47e57567d4a18f/Companies.csv')

company_info_df = pd.read_csv(data)


Now let's use DuckDB to allow working with on-disk and in memory data.

In [12]:
con = duckdb.connect('local.db')
con.sql("""CREATE TABLE IF NOT EXISTS company_data AS
           SELECT *
           FROM company_data_df""")
con.sql("""create table if not exists company_ceos as select * from company_ceos_df""")
con.sql("""create table if not exists executives as select * from exec_df""")

# query the table
con.table("company_data").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────┬────────────────────────────────────────────────┬────────────────────┬──────────────┬─────────────────────────────────────┬─────────────┬────────────────────────────────────────┬────────────────┬─────────────────────────────────────────────────────────────────┬───────────────────────────┬─────────────────────────┐
│ Unnamed: 0 │                      name                      │       domain       │ year founded │              industry               │ size range  │                locality                │    country     │                          linkedin url                           │ current employee estimate │ total employee estimate │
│   int64    │                    varchar                     │      varchar       │    double    │               varchar               │   varchar   │                varchar                 │    varchar     │                             varchar                             │           int64           │          int64          │
├─────────

# 2.0 Data Transformation and Querying

Looking at our data to clean via *projection*...

Generally, we can extract one "narrower" table form another by using **double brackets**.

In [13]:
# Let's take a look at the data.  Here's a way of PROJECTING the exec_df dataframe into
# a smaller table

exec_df[['name', 'born']]

,name,born
0,Julie_Sweet,NaT
1,Kumar_Mangalam_Birla,1967-06-14 00:00:00
2,Shantanu_Narayen,1963-05-27 00:00:00
3,Guillaume_Faury,1968-02-22 00:00:00
4,Eddie_Wu,NaT
...,...,...
133,Ann_Sarnoff,NaT
134,Jason_Kilar,1971-04-26 00:00:00
135,Charles_Scharf,1965-04-24 00:00:00
136,John_Mackey,NaT


In [14]:
# In SQL it's SELECT with the fields FROM the table
con.sql('select name, born from executives')

┌──────────────────────┬─────────────────────┐
│         name         │        born         │
│       varchar        │      timestamp      │
├──────────────────────┼─────────────────────┤
│ Julie_Sweet          │ NULL                │
│ Kumar_Mangalam_Birla │ 1967-06-14 00:00:00 │
│ Shantanu_Narayen     │ 1963-05-27 00:00:00 │
│ Guillaume_Faury      │ 1968-02-22 00:00:00 │
│ Eddie_Wu             │ NULL                │
│ Andy_Jassy           │ 1968-01-13 00:00:00 │
│ Lisa_Su              │ NULL                │
│ Stephen_Squeri       │ NULL                │
│ Joseph_R._Swedish    │ 1951-05-17 00:00:00 │
│ Tim_Cook             │ 1960-11-01 00:00:00 │
│    ·                 │          ·          │
│    ·                 │          ·          │
│    ·                 │          ·          │
│ Vittorio_Colao       │ 1961-10-03 00:00:00 │
│ Herbert_Diess        │ 1958-10-24 00:00:00 │
│ Robert_Iger          │ 1951-02-10 00:00:00 │
│ Stefano_Pessina      │ 1941-06-04 00:00:00 │
│ Doug_McMill

In [15]:
# If I use single brackets, I can extract a single column as a Series.
exec_df['name']

,name
0,Julie_Sweet
1,Kumar_Mangalam_Birla
2,Shantanu_Narayen
3,Guillaume_Faury
4,Eddie_Wu
...,...
133,Ann_Sarnoff
134,Jason_Kilar
135,Charles_Scharf
136,John_Mackey


In [16]:
# We can use SQL over the dataframe OR here
duckdb.sql('SELECT name FROM exec_df')

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ Julie_Sweet          │
│ Kumar_Mangalam_Birla │
│ Shantanu_Narayen     │
│ Guillaume_Faury      │
│ Eddie_Wu             │
│ Andy_Jassy           │
│ Lisa_Su              │
│ Stephen_Squeri       │
│ Joseph_R._Swedish    │
│ Tim_Cook             │
│    ·                 │
│    ·                 │
│    ·                 │
│ Vittorio_Colao       │
│ Herbert_Diess        │
│ Robert_Iger          │
│ Stefano_Pessina      │
│ Doug_McMillon        │
│ Ann_Sarnoff          │
│ Jason_Kilar          │
│ Charles_Scharf       │
│ John_Mackey          │
│ Rich_Barton          │
├──────────────────────┤
│ 138 rows (20 shown)  │
└──────────────────────┘

In [17]:
# Notice anything awry?

for person in exec_df['name']:
    print (person)

Julie_Sweet
Kumar_Mangalam_Birla
Shantanu_Narayen
Guillaume_Faury
Eddie_Wu
Andy_Jassy
Lisa_Su
Stephen_Squeri
Joseph_R._Swedish
Tim_Cook
Aditya_Mittal
John_Stankey
Charles_Woodburn
Tapan_Singhel
Carlos_Torres_Vila
Brian_Moynihan
C.S._Venkatakrishnan
Warren_Buffett
Hubert_Joly
Sunil_Bharti_Mittal
Stephen_A._Schwarzman
Mike_Henry
Oliver_Zipse
Kelly_Ortberg
Rich_Lesser
Bob_Dudley
Hock_Tan
Denise_Morrison
Mark_Shuttleworth
Richard_Fairbank
Jim_Umpleby
Evan_Greenberg
Chuck_Robbins
Jane_Fraser
James_Quincey
Michael_J._Cavanagh
Thomas_Gottstein
Ola_Källenius
Michael_Dell
Ed_Bastian
Christian_Sewing
Tobias_Meyer
Edward_D._Breen
Devin_Wenig
Börje_Ekholm
Darren_Woods
Carmine_Di_Sibio
Mark_Zuckerberg
Raj_Subramaniam
Sergio_Marchionne
Abigail_Johnson
James_Hackett
Terry_Gou
Lachlan_Murdoch
Ryan_Cohen
Phebe_Novakovic
H._Lawrence_Culp_Jr.
Mary_T._Barra
Emma_Walmsley
David_M._Solomon
Sundar_Pichai
C_Vijayakumar
Antonio_Neri
Darius_Adamczyk
Noel_Quinn
Arvind_Krishna
Salil_Parekh
Lip-Bu_Tan
James_Dimon


In [18]:
def to_space(x):
  return x.replace('_', ' ')

# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(to_space)

,name
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
133,Ann Sarnoff
134,Jason Kilar
135,Charles Scharf
136,John Mackey


In [19]:
# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(lambda x: x.replace('_', ' '))

,name
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
133,Ann Sarnoff
134,Jason Kilar
135,Charles Scharf
136,John Mackey


In [20]:
# I can also use *apply* to call a function over the rows of a dataframe
exec_df.apply(lambda x: x['name'].replace('_', ' '), axis=1)

,0
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
133,Ann Sarnoff
134,Jason Kilar
135,Charles Scharf
136,John Mackey


In [21]:
# Let's clean the name by removing underscores...
exec_df['clean_name'] = exec_df['name'].apply(lambda x: x.replace('_', ' '))

exec_df

,name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,NaT,Julie Sweet
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_B...,1967-06-14 00:00:00,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27 00:00:00,Shantanu Narayen
3,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22 00:00:00,Guillaume Faury
4,Eddie_Wu,https://en.wikipedia.org/wiki/Eddie_Wu,NaT,Eddie Wu
...,...,...,...,...
133,Ann_Sarnoff,https://en.wikipedia.org/wiki/Ann_Sarnoff,NaT,Ann Sarnoff
134,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26 00:00:00,Jason Kilar
135,Charles_Scharf,https://en.wikipedia.org/wiki/Charles_Scharf,1965-04-24 00:00:00,Charles Scharf
136,John_Mackey,https://en.wikipedia.org/wiki/John_Mackey,NaT,John Mackey


In [22]:
exec_df.rename(columns={'name': 'old_name'})

,old_name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,NaT,Julie Sweet
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_B...,1967-06-14 00:00:00,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27 00:00:00,Shantanu Narayen
3,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22 00:00:00,Guillaume Faury
4,Eddie_Wu,https://en.wikipedia.org/wiki/Eddie_Wu,NaT,Eddie Wu
...,...,...,...,...
133,Ann_Sarnoff,https://en.wikipedia.org/wiki/Ann_Sarnoff,NaT,Ann Sarnoff
134,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26 00:00:00,Jason Kilar
135,Charles_Scharf,https://en.wikipedia.org/wiki/Charles_Scharf,1965-04-24 00:00:00,Charles Scharf
136,John_Mackey,https://en.wikipedia.org/wiki/John_Mackey,NaT,John Mackey


In [23]:
# We can do the same via SQL.

duckdb.sql("""SELECT name, replace(name, '_', ' ') AS clean_name
              FROM exec_df""")

┌──────────────────────┬──────────────────────┐
│         name         │      clean_name      │
│       varchar        │       varchar        │
├──────────────────────┼──────────────────────┤
│ Julie_Sweet          │ Julie Sweet          │
│ Kumar_Mangalam_Birla │ Kumar Mangalam Birla │
│ Shantanu_Narayen     │ Shantanu Narayen     │
│ Guillaume_Faury      │ Guillaume Faury      │
│ Eddie_Wu             │ Eddie Wu             │
│ Andy_Jassy           │ Andy Jassy           │
│ Lisa_Su              │ Lisa Su              │
│ Stephen_Squeri       │ Stephen Squeri       │
│ Joseph_R._Swedish    │ Joseph R. Swedish    │
│ Tim_Cook             │ Tim Cook             │
│    ·                 │    ·                 │
│    ·                 │    ·                 │
│    ·                 │    ·                 │
│ Vittorio_Colao       │ Vittorio Colao       │
│ Herbert_Diess        │ Herbert Diess        │
│ Robert_Iger          │ Robert Iger          │
│ Stefano_Pessina      │ Stefano Pessina

## 2.1. Selecting a subset of the rows

In [24]:
# Here's a column

exec_df['clean_name']

,clean_name
0,Julie Sweet
1,Kumar Mangalam Birla
2,Shantanu Narayen
3,Guillaume Faury
4,Eddie Wu
...,...
133,Ann Sarnoff
134,Jason Kilar
135,Charles Scharf
136,John Mackey


In [25]:
# We can apply a test (predicate) to each column, returning a Series of boolean true/false values

exec_df['clean_name'] == 'Julie Sweet'

,clean_name
0,True
1,False
2,False
3,False
4,False
...,...
133,False
134,False
135,False
136,False


In [26]:
# If we compose these, we'll get only those rows where the boolean condition was True

exec_df[exec_df['clean_name'] == 'Julie Sweet']

,name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,NaT,Julie Sweet


SQL lets us use any case, but convention is to capitalize the SQL keywords such as `SELECT`, `FROM`, `WHERE` to aid in readability.  Also, we should use single-quotes for SQL strings, so we'll typically pass the SQL command in with double-quotes.

In [27]:
duckdb.sql("SELECT * FROM exec_df WHERE clean_name='Julie Sweet'")

┌─────────────┬───────────────────────────────────────────┬───────────┬─────────────┐
│    name     │                   page                    │   born    │ clean_name  │
│   varchar   │                  varchar                  │ timestamp │   varchar   │
├─────────────┼───────────────────────────────────────────┼───────────┼─────────────┤
│ Julie_Sweet │ https://en.wikipedia.org/wiki/Julie_Sweet │ NULL      │ Julie Sweet │
└─────────────┴───────────────────────────────────────────┴───────────┴─────────────┘

In [28]:
exec_df[exec_df['clean_name'] == 'Julie Sweet'][['page']]

,page
0,https://en.wikipedia.org/wiki/Julie_Sweet



Here we'll use the triple-quote syntax for Python strings, which allows us to pass a multi-line string to SQL...


In [29]:
duckdb.sql("""SELECT clean_name
            FROM exec_df
            WHERE clean_name='Julie Sweet'""")

┌─────────────┐
│ clean_name  │
│   varchar   │
├─────────────┤
│ Julie Sweet │
└─────────────┘

In [30]:
import numpy as np

exec_df.dropna(subset=['born'])

,name,page,born,clean_name
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_B...,1967-06-14 00:00:00,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27 00:00:00,Shantanu Narayen
3,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22 00:00:00,Guillaume Faury
5,Andy_Jassy,https://en.wikipedia.org/wiki/Andy_Jassy,1968-01-13 00:00:00,Andy Jassy
8,Joseph_R._Swedish,https://en.wikipedia.org/wiki/Joseph_R._Swedish,1951-05-17 00:00:00,Joseph R. Swedish
...,...,...,...,...
130,Robert_Iger,https://en.wikipedia.org/wiki/Robert_Iger,1951-02-10 00:00:00,Robert Iger
131,Stefano_Pessina,https://en.wikipedia.org/wiki/Stefano_Pessina,1941-06-04 00:00:00,Stefano Pessina
132,Doug_McMillon,https://en.wikipedia.org/wiki/Doug_McMillon,1966-10-17 00:00:00,Doug McMillon
134,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26 00:00:00,Jason Kilar


In [31]:
duckdb.sql("""SELECT *
            FROM exec_df
            WHERE born IS NOT NULL""")

┌──────────────────────┬────────────────────────────────────────────────────┬─────────────────────┬──────────────────────┐
│         name         │                        page                        │        born         │      clean_name      │
│       varchar        │                      varchar                       │      timestamp      │       varchar        │
├──────────────────────┼────────────────────────────────────────────────────┼─────────────────────┼──────────────────────┤
│ Kumar_Mangalam_Birla │ https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla │ 1967-06-14 00:00:00 │ Kumar Mangalam Birla │
│ Shantanu_Narayen     │ https://en.wikipedia.org/wiki/Shantanu_Narayen     │ 1963-05-27 00:00:00 │ Shantanu Narayen     │
│ Guillaume_Faury      │ https://en.wikipedia.org/wiki/Guillaume_Faury      │ 1968-02-22 00:00:00 │ Guillaume Faury      │
│ Andy_Jassy           │ https://en.wikipedia.org/wiki/Andy_Jassy           │ 1968-01-13 00:00:00 │ Andy Jassy           │
│ Joseph_R._Swed

## 2.2. Joining Data

We start with a simple join between company_ceos_df and exec_df and persist it to the database.  We then check how many companies did not have a match on CEO name.

In [32]:
exec_df[['clean_name', 'born']]

,clean_name,born
0,Julie Sweet,NaT
1,Kumar Mangalam Birla,1967-06-14 00:00:00
2,Shantanu Narayen,1963-05-27 00:00:00
3,Guillaume Faury,1968-02-22 00:00:00
4,Eddie Wu,NaT
...,...,...
133,Ann Sarnoff,NaT
134,Jason Kilar,1971-04-26 00:00:00
135,Charles Scharf,1965-04-24 00:00:00
136,John Mackey,NaT


In [33]:
# Remove any duplicate executive entries

exec_df = exec_df.drop_duplicates()

In [34]:
company_ceos_df[['Executive', 'Company']]

,Executive,Company
0,Julie Sweet,Accenture
1,Kumar Mangalam Birla,Aditya Birla Group
2,Shantanu Narayen,Adobe Systems
3,Guillaume Faury,Airbus
4,Eddie Wu,Alibaba
...,...,...
133,Ann Sarnoff,Warner Brothers
134,Jason Kilar,WarnerMedia
135,Charles Scharf,Wells Fargo
136,John Mackey,Whole Foods Market


In [35]:
company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'])

,Executive,Company,clean_name,born
0,Julie Sweet,Accenture,Julie Sweet,NaT
1,Kumar Mangalam Birla,Aditya Birla Group,Kumar Mangalam Birla,1967-06-14 00:00:00
2,Shantanu Narayen,Adobe Systems,Shantanu Narayen,1963-05-27 00:00:00
3,Guillaume Faury,Airbus,Guillaume Faury,1968-02-22 00:00:00
4,Eddie Wu,Alibaba,Eddie Wu,NaT
...,...,...,...,...
133,Ann Sarnoff,Warner Brothers,Ann Sarnoff,NaT
134,Jason Kilar,WarnerMedia,Jason Kilar,1971-04-26 00:00:00
135,Charles Scharf,Wells Fargo,Charles Scharf,1965-04-24 00:00:00
136,John Mackey,Whole Foods Market,John Mackey,NaT


We can `JOIN ON` in the `FROM` clause.

In [36]:
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df JOIN exec_df ON Executive=clean_name
          """)

┌──────────────────────┬──────────────────────────┬─────────────────────┐
│      Executive       │         Company          │        born         │
│       varchar        │         varchar          │      timestamp      │
├──────────────────────┼──────────────────────────┼─────────────────────┤
│ Julie Sweet          │ Accenture                │ NULL                │
│ Kumar Mangalam Birla │ Aditya Birla Group       │ 1967-06-14 00:00:00 │
│ Shantanu Narayen     │ Adobe Systems            │ 1963-05-27 00:00:00 │
│ Guillaume Faury      │ Airbus                   │ 1968-02-22 00:00:00 │
│ Eddie Wu             │ Alibaba                  │ NULL                │
│ Andy Jassy           │ Amazon                   │ 1968-01-13 00:00:00 │
│ Lisa Su              │ Advanced Micro Devices   │ NULL                │
│ Stephen Squeri       │ American Express         │ NULL                │
│ Joseph R. Swedish    │ Anthem                   │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple        

Note there is another way you'll sometimes see, in older versions of SQL... Which is to put the join as a `WHERE` condition:

In [37]:
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df, exec_df
            WHERE Executive=clean_name
          """)

┌──────────────────────┬──────────────────────────┬─────────────────────┐
│      Executive       │         Company          │        born         │
│       varchar        │         varchar          │      timestamp      │
├──────────────────────┼──────────────────────────┼─────────────────────┤
│ Julie Sweet          │ Accenture                │ NULL                │
│ Kumar Mangalam Birla │ Aditya Birla Group       │ 1967-06-14 00:00:00 │
│ Shantanu Narayen     │ Adobe Systems            │ 1963-05-27 00:00:00 │
│ Guillaume Faury      │ Airbus                   │ 1968-02-22 00:00:00 │
│ Eddie Wu             │ Alibaba                  │ NULL                │
│ Andy Jassy           │ Amazon                   │ 1968-01-13 00:00:00 │
│ Lisa Su              │ Advanced Micro Devices   │ NULL                │
│ Stephen Squeri       │ American Express         │ NULL                │
│ Joseph R. Swedish    │ Anthem                   │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple        

OK, let's drop the cases where we don't have a CEO's birthday: these aren't useful!

In [38]:
# Shall we skip the cases where we don't have the birthday?
duckdb.sql("""
            SELECT Executive, Company, born
            FROM company_ceos_df JOIN exec_df ON Executive=clean_name
            WHERE born is not null
          """)

┌──────────────────────┬─────────────────────────────────┬─────────────────────┐
│      Executive       │             Company             │        born         │
│       varchar        │             varchar             │      timestamp      │
├──────────────────────┼─────────────────────────────────┼─────────────────────┤
│ Kumar Mangalam Birla │ Aditya Birla Group              │ 1967-06-14 00:00:00 │
│ Shantanu Narayen     │ Adobe Systems                   │ 1963-05-27 00:00:00 │
│ Guillaume Faury      │ Airbus                          │ 1968-02-22 00:00:00 │
│ Andy Jassy           │ Amazon                          │ 1968-01-13 00:00:00 │
│ Joseph R. Swedish    │ Anthem                          │ 1951-05-17 00:00:00 │
│ Tim Cook             │ Apple                           │ 1960-11-01 00:00:00 │
│ Aditya Mittal        │ Arcelor Mittal                  │ 1976-01-22 00:00:00 │
│ Charles Woodburn     │ BAE Systems                     │ 1971-03-11 00:00:00 │
│ Tapan Singhel        │ Baj

In [39]:
company_ceos_df[['Executive', 'Company']]

,Executive,Company
0,Julie Sweet,Accenture
1,Kumar Mangalam Birla,Aditya Birla Group
2,Shantanu Narayen,Adobe Systems
3,Guillaume Faury,Airbus
4,Eddie Wu,Alibaba
...,...,...
133,Ann Sarnoff,Warner Brothers
134,Jason Kilar,WarnerMedia
135,Charles Scharf,Wells Fargo
136,John Mackey,Whole Foods Market


In [40]:
exec_df[['clean_name', 'born']]

,clean_name,born
0,Julie Sweet,NaT
1,Kumar Mangalam Birla,1967-06-14 00:00:00
2,Shantanu Narayen,1963-05-27 00:00:00
3,Guillaume Faury,1968-02-22 00:00:00
4,Eddie Wu,NaT
...,...,...
133,Ann Sarnoff,NaT
134,Jason Kilar,1971-04-26 00:00:00
135,Charles Scharf,1965-04-24 00:00:00
136,John Mackey,NaT


## 2.4. Finding the misses in the join with OUTER JOINs.

Note that the join above resulted in 174 rows.  However, there are more rows in company_ceos_df so we are missing some companies.  We can see which are missed using a LEFT OUTERJOIN (aka LEFT JOIN); setting "indicator= True" allows us to see which tuples in company_ceos_df failed to find a match (left_only, e.g. row 24 and 172).

In [ ]:
pd.set_option('display.max_rows', 200)
display(company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'], how="left", indicator=True))



In [ ]:
pd.set_option('display.max_rows', 50)
result_df = company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']],
                                                left_on=['Executive'],
                                                right_on=['clean_name'], how="outer", indicator=True)

result_df[result_df['_merge'] != 'both']


We can also do this in SQL (there is no indicator but we can test for NULL):

In [ ]:
duckdb.sql("""
            SELECT Executive, Company, clean_name, born
            FROM company_ceos_df FULL JOIN exec_df ON Executive=clean_name
            WHERE clean_name is null or Company is null
          """)

## 2.3. Composing Joins

Of course, we can join the results of a join with another table -- representing a *composition*!



Let's join with company data!

In [ ]:
duckdb.sql("""SELECT Executive, Company, born
            FROM company_ceos_df
            JOIN exec_df ON Executive=replace(name, '_', ' ')
            JOIN company_data_df cd ON Company=cd.name
            WHERE born is not null""")

Hmm, what is wrong here?

Let's switch to the tables saved in DuckDB for these.

In [ ]:
con.sql('SELECT * from company_data')

Notice the case for `name`?

In [ ]:
con.sql("""SELECT Executive, Company, born
            FROM company_ceos
            JOIN executives ON Executive=replace(name, '_', ' ')
            JOIN company_data cd ON lower(Company)=lower(cd.name)
            WHERE born is not null
            ORDER BY Company""")

Hmm, there are duplicates!  This is because of fields in the `company_data` table that we don't care about. We can remove the duplicates via `SELECT DISTINCT`.

In [ ]:
con.sql("""SELECT DISTINCT Executive, Company, born
            FROM company_ceos
            JOIN executives ON Executive=replace(name, '_', ' ')
            JOIN company_data cd ON lower(Company)=lower(cd.name)
            WHERE born is not null
            ORDER BY Company""")

Can we do all of this in Pandas? Of course!

First, we need to lowercase the company names.

In [ ]:
company_ceos_df['company_lc'] = company_ceos_df['Company'].apply(lambda x: x.lower())

Notice this is slower than DuckDB?

In [ ]:
company_ceos_df.merge(exec_df.dropna(),
                      left_on=['Executive'],
                      right_on=['clean_name']).\
                      merge(company_data_df,
                            left_on='company_lc',
                            right_on='name')[['Executive','Company','born']].drop_duplicates().sort_values('Company')

## Section 2 Exercises

Hmm, there seem to be quite a few Roberts and Bobs. (Maybe also Roberta?)

Using either Pandas operations like `merge` and the various bracket notations (but **not** the `loc`/`iloc` commands if you know these): write a query that takes the `company_ceos_df` and returns all companies that are overseen by someone with names `Bob`, `Robert`, or `Roberta`.

First, define a function that *robustly* returns whether we have a "bob" entry (i.e., "Bob", "Robert", or "Roberta" as a string, in any case
:

In [ ]:
def bob(name: str) -> bool:
  first_name = name # TODO: change this to get first name!

  return False # TODO: change this!

In [ ]:
from dill.source import getsource
grader.grade('bob_test', getsource(bob))

Now *select a subset of the rows* that have Bob/etc. using the function `bob` and the `apply` function.

In [ ]:
bob_co_df = None# TODO
bob_co_df

In [ ]:
grader.grade(test_case_id='bobs_your_uncle_or_aunt', answer=bob_co_df)